In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, RobustScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
path = 'C:/Users/PC/Desktop/Master_IA/Principios_Machine_Learning/Datos_Etapa_1_csv.csv'
with open(path, 'r') as file:
    content = file.read()

content = content.replace('"', '')  # Eliminar las comillas dobles

with open(path, 'w') as file:
    file.write(content)

In [3]:
#Carga y verificacion del CSV 
data_original = pd.read_csv(path, sep=',')
data_copy = data_original.copy()
print(data_copy.shape)
data_copy.head()


(17379, 9)


,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0,1,Night


In [4]:
#eliminar columnas categoricas mientras se construye el modelo
data_copy=data_copy.drop(['season','weathersit','time_of_day'], axis=1)
print(data_copy.shape)
data_copy.head()

(17379, 6)


,weekday,temp,atemp,hum,windspeed,cnt
0,6,3.28,3.0014,0.81,0.0,16
1,6,2.34,1.9982,0.80,0.0,40
2,6,2.34,1.9982,0.80,0.0,32
3,6,3.28,3.0014,0.75,0.0,13
4,6,3.28,3.0014,0.75,0.0,1


In [5]:
#verificar data incompleta
data_copy.isna().sum()

weekday      0
temp         0
atemp        0
hum          0
windspeed    0
cnt          0
dtype: int64

In [6]:
#verificar registros duplicados 
data_copy.duplicated().sum()

np.int64(71)

In [7]:
#eliminar registros replicados
data_copy = data_copy.drop_duplicates()
print(data_copy.shape)

(17308, 6)


In [8]:
#SEPARCION DE VARIABLE OBJETIVO DE INDEPENDIENTES   ---------------------------------------------- REGRESION CON REGULACION LASSO
train_lasso, test_lasso = train_test_split(data_copy, test_size=0.2, random_state=77)
x_train_lasso = train_lasso.drop(['cnt'], axis = 1)
y_train_lasso = train_lasso['cnt']
#ESTANDARIZACIÓN DE LOS DATOS
columns = x_train_lasso.columns  
#creacion de objeto StandardScaler()
scaler = StandardScaler()
x_train_lasso = pd.DataFrame(scaler.fit_transform(x_train_lasso), columns = columns)
#BUSQUEDA DE HIPERPARAMETRO Y ENTRENAMIENTO
kfold = KFold(n_splits=10, shuffle= True, random_state=0)
#creacion objeto clase lasso
lasso = Lasso()
#definición de busqueda de hiperparametro
valores_alpha = [1, 2, 3, 4, 5]
param_grid_lasso = {'alpha': valores_alpha}
grid_lasso = GridSearchCV(lasso, param_grid_lasso, cv = kfold, n_jobs = 1, scoring= 'neg_mean_squared_error')
grid_lasso.fit(x_train_lasso, y_train_lasso)


GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Lasso(), n_jobs=1, param_grid={'alpha': [1, 2, 3, 4, 5]},
             scoring='neg_mean_squared_error')

In [9]:
#mostrar mejor valor alfa 
mejor_modelo_lasso = grid_lasso.best_estimator_
x_test_lasso = test_lasso.drop(['cnt'], axis = 1)
y_test_lasso = test_lasso['cnt']
x_test_lasso = pd.DataFrame(scaler.transform(x_test_lasso), columns= columns)

print("mejor parametro lasso:", grid_lasso.best_params_)
list(zip(x_train_lasso.columns, mejor_modelo_lasso.coef_))

mejor parametro lasso: {'alpha': 1}


[('weekday', np.float64(2.4527225007437066)),
 ('temp', np.float64(24.257855170472723)),
 ('atemp', np.float64(45.7116215010748)),
 ('hum', np.float64(-51.9817641094759)),
 ('windspeed', np.float64(4.237435831596017))]

In [10]:
#predicciones
y_pred_lasso = mejor_modelo_lasso.predict(x_test_lasso)
#manejor de errores
rmse_lasso = mean_squared_error(y_test_lasso, y_pred_lasso, squared = False)
mae_lasso = mean_absolute_error(y_test_lasso, y_pred_lasso)
r2_lasso = r2_score(y_test_lasso, y_pred_lasso)

print(y_pred_lasso)
print(["RMSE:", rmse_lasso], ["MAE:", mae_lasso], ["R2:", r2_lasso])

[229.05046091 284.33900372  99.41829697 ... 204.77070999 102.01075147
 131.66635311]
['RMSE:', np.float64(153.6197870886861)] ['MAE:', np.float64(114.8682317240207)] ['R2:', 0.25556347828063286]


C:\Users\PC\IdeaProjects\python\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
#SEPARCION DE VARIABLE OBJETIVO DE INDEPENDIENTES Y SEPARACIÓN TEST ENTRENAMIENTO ---------------------------------------------- POLINOMIAL
x_pol = data_copy.drop(['cnt'], axis=1)
y_pol = data_copy['cnt']
x_train_pol, x_test_pol, y_train_pol, y_test_pol = train_test_split(x_pol, y_pol, test_size=0.2, random_state=77)
#BUSQUEDA DE HIPER PARAMETRO
polynomial_regression = make_pipeline(
    PolynomialFeatures(),
    RobustScaler(),
    LinearRegression()
)
kfold = KFold(n_splits=10, shuffle= True, random_state=0)
#espacio de busqueda 
valores_busqueda = [2,3]
param_grid_pol = {'polynomialfeatures__degree': valores_busqueda}
grid_pol = GridSearchCV(polynomial_regression, param_grid_pol, cv = kfold, n_jobs = 1, scoring= 'neg_mean_squared_error')
grid_pol.fit(x_train_pol, y_train_pol)
print("mejor parametro polinomial: ", grid_pol.best_params_)

mejor parametro polinomial:  {'polynomialfeatures__degree': 3}


In [12]:
mejor_modelo_pol = grid_pol.best_estimator_
#prediciones
y_pred_pol = mejor_modelo_pol.predict(x_test_pol)
#manejo de errores
rmse_pol = mean_squared_error(y_test_pol, y_pred_pol, squared = False)
mae_pol = mean_absolute_error(y_test_pol, y_pred_pol)
r2_pol = r2_score(y_test_pol, y_pred_pol)
print(y_pred_pol)
print(["RMSE:", rmse_pol], ["MAE",mae_pol], ["R2:", r2_pol])

[284.09263167 298.85033325  76.55867342 ... 228.64383521 113.68197709
  20.95034722]
['RMSE:', np.float64(149.5871277925923)] ['MAE', np.float64(110.85520414728232)] ['R2:', 0.29413475254745436]


C:\Users\PC\IdeaProjects\python\venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [13]:
print("ERRORES LASSO:")
print(["RMSE:", rmse_lasso], ["MAE",mae_lasso], ["R2:", r2_lasso])
print("ERRORES POLYNOMIAL:")
print(["RMSE:", rmse_pol], ["MAE:", mae_pol], ["R2:", r2_pol])

ERRORES LASSO:
['RMSE:', np.float64(153.6197870886861)] ['MAE', np.float64(114.8682317240207)] ['R2:', 0.25556347828063286]
ERRORES POLYNOMIAL:
['RMSE:', np.float64(149.5871277925923)] ['MAE:', np.float64(110.85520414728232)] ['R2:', 0.29413475254745436]
